[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_oil_acf_pacf/EMQA_oil_acf_pacf.ipynb)

# EMQA_oil_acf_pacf

**Oil Returns ACF/PACF for Order Selection**

ACF and PACF plots of Brent oil log returns with 40 lags and 95% confidence intervals to guide ARIMA order selection.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from statsmodels.tsa.stattools import acf, pacf

brent = fetch('BZ=F')
log_ret = np.log(brent / brent.shift(1)).dropna()

nlags = 40
acf_vals = acf(log_ret, nlags=nlags)
pacf_vals = pacf(log_ret, nlags=nlags)
conf = 1.96 / np.sqrt(len(log_ret))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF
axes[0].bar(range(nlags + 1), acf_vals, width=0.4, color=COLORS['blue'], alpha=0.85)
axes[0].axhline(conf, color=COLORS['red'], linestyle='--', linewidth=0.8, label='95% CI')
axes[0].axhline(-conf, color=COLORS['red'], linestyle='--', linewidth=0.8)
axes[0].axhline(0, color='black', linewidth=0.5)
axes[0].set_title('ACF of Brent Log Returns', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

# Count significant ACF lags (excluding lag 0)
sig_acf = [k for k in range(1, nlags + 1) if abs(acf_vals[k]) > conf]
acf_note = f"Significant lags: {sig_acf[:5]}..." if len(sig_acf) > 5 else f"Significant lags: {sig_acf}"
props = dict(boxstyle='round,pad=0.4', facecolor='lightyellow', alpha=0.8)
axes[0].text(0.98, 0.95, f"Suggested q (MA order)\n{acf_note}",
             transform=axes[0].transAxes, fontsize=9, ha='right', va='top', bbox=props)

# PACF
axes[1].bar(range(nlags + 1), pacf_vals, width=0.4, color=COLORS['green'], alpha=0.85)
axes[1].axhline(conf, color=COLORS['red'], linestyle='--', linewidth=0.8, label='95% CI')
axes[1].axhline(-conf, color=COLORS['red'], linestyle='--', linewidth=0.8)
axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].set_title('PACF of Brent Log Returns', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('PACF')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

sig_pacf = [k for k in range(1, nlags + 1) if abs(pacf_vals[k]) > conf]
pacf_note = f"Significant lags: {sig_pacf[:5]}..." if len(sig_pacf) > 5 else f"Significant lags: {sig_pacf}"
axes[1].text(0.98, 0.95, f"Suggested p (AR order)\n{pacf_note}",
             transform=axes[1].transAxes, fontsize=9, ha='right', va='top', bbox=props)

fig.suptitle('Brent Oil — ACF / PACF Order Selection', fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'oil_acf_pacf_selection.pdf')
plt.show()
